# Notebook for initial data cleanup
## Reads CSV data and performs the following:
- Finds means, standard deviations and standard error for triplicate measurements
- Aggregates values and adds appropriate columns for mean, std, and se (e.g. anthos = anthos_mean, anthos_std, anthos_se)
- Pickles data for easier access down the line
- Performs this process for both Cab Sauv (cs) and Petit Verdot (pv) datasets

In [1]:
#For dataframes
import pandas as pd

#For math stuff
import numpy as np

### Start with Cab Sauv (cs) data, and come up with a script to clean up the dataframe

In [2]:
#read in tannin data csv
data=pd.read_csv("../01-input_data/tannin_data-cs.csv")

In [3]:
data.head(5)

,varietal,date,id,ph,treat,dpt,anthos,spp,lpp,tannins,phenolics,non_tannin
0,CS,6/20/2023,ACET-3.25,3.25,ACET,0,266,0.99,0.84,431,1278,847
1,CS,6/20/2023,ACET-3.25,3.25,ACET,0,219,0.68,1.23,358,1023,665
2,CS,6/20/2023,ACET-3.25,3.25,ACET,0,213,0.64,1.17,375,1036,660
3,CS,11/28/2023,ACET-3.25,3.25,ACET,161,0,1.18,3.87,370,1081,711
4,CS,11/28/2023,ACET-3.25,3.25,ACET,161,0,1.18,3.90,341,1010,669


In [4]:
#Create a new column of unique names so we can average them and find SE
data['group']=data['id'].astype('str') + '-' + data['dpt'].astype('str')
data.head(5)

,varietal,date,id,ph,treat,dpt,anthos,spp,lpp,tannins,phenolics,non_tannin,group
0,CS,6/20/2023,ACET-3.25,3.25,ACET,0,266,0.99,0.84,431,1278,847,ACET-3.25-0
1,CS,6/20/2023,ACET-3.25,3.25,ACET,0,219,0.68,1.23,358,1023,665,ACET-3.25-0
2,CS,6/20/2023,ACET-3.25,3.25,ACET,0,213,0.64,1.17,375,1036,660,ACET-3.25-0
3,CS,11/28/2023,ACET-3.25,3.25,ACET,161,0,1.18,3.87,370,1081,711,ACET-3.25-161
4,CS,11/28/2023,ACET-3.25,3.25,ACET,161,0,1.18,3.90,341,1010,669,ACET-3.25-161


### Aggregate and add summary statistics

In [5]:
#Create new dataframe that summarizes entries in to means and std_error

#Identify columns to aggregate
cols=['anthos','spp','lpp','tannins','phenolics','non_tannin']

#Create new dataframe, aggregate columns with summary statistics 'mean', 'stdev', and 'counts'
means=pd.DataFrame()
means=data.groupby('group')[cols].agg(['mean','std','count']).reset_index()

#Get rid of the multi-level indexing
means.columns=['_'.join(col).strip() for col in means.columns.values]

#Add standard error columns
for col in cols:
    means[f'{col}_SE']=means[f'{col}_std']/np.sqrt(means[f'{col}_count'])

#Get rid of underscore in 'group' column
means=means.rename(columns={'group_':'group'})

In [6]:
print(means)

            group  anthos_mean  anthos_std  anthos_count  spp_mean   spp_std  \
0     ACET-3.25-0   232.666667   29.022979             3  0.770000  0.191572   
1   ACET-3.25-161     0.000000    0.000000             3  1.126667  0.092376   
2   ACET-3.25-282    31.333333    5.131601             3  1.126667  0.156312   
3   ACET-3.25-529    34.333333    6.658328             3  1.406667  0.106927   
4     ACET-3.50-0   232.666667   29.022979             3  0.770000  0.191572   
..            ...          ...         ...           ...       ...       ...   
59   SO2-3.75-529   105.000000   19.974984             3  1.096667  0.143643   
60     SO2-4.00-0   208.333333    4.932883             3  0.690000  0.096437   
61   SO2-4.00-161   145.666667   27.098585             3  0.560000  0.140000   
62   SO2-4.00-282   154.333333   24.419937             3  0.623333  0.076376   
63   SO2-4.00-529   125.666667   11.503623             3  1.046667  0.214554   

    spp_count  lpp_mean   lpp_std  lpp_

In [7]:
#split the groups back up in to treatment, ph, and days post-treatment
means[['trt','ph','dpt']]=means['group'].str.split('-', expand=True)

In [8]:
#Drop 'dpt' entries for dpt=0
means=means[means['dpt'] != '0']

In [9]:
#Make sure pH's are numerical values and not strings
means['ph']=pd.to_numeric(means['ph'])

In [10]:
print(means)

            group  anthos_mean  anthos_std  anthos_count  spp_mean   spp_std  \
1   ACET-3.25-161     0.000000    0.000000             3  1.126667  0.092376   
2   ACET-3.25-282    31.333333    5.131601             3  1.126667  0.156312   
3   ACET-3.25-529    34.333333    6.658328             3  1.406667  0.106927   
5   ACET-3.50-161     0.000000    0.000000             3  1.016667  0.155671   
6   ACET-3.50-282    34.666667   11.846237             3  1.100000  0.310966   
7   ACET-3.50-529    40.333333    9.712535             3  1.553333  0.281839   
9   ACET-3.75-161    10.333333   17.897858             3  1.090000  0.321403   
10  ACET-3.75-282    40.666667   21.548395             3  1.266667  0.283608   
11  ACET-3.75-529    41.000000   25.514702             3  1.896667  0.277369   
13  ACET-4.00-161    11.333333    4.163332             3  1.116667  0.076376   
14  ACET-4.00-282    54.666667    8.082904             3  1.276667  0.115902   
15  ACET-4.00-529    74.666667   10.4083

# Create a function to make this easier

In [11]:
def cleanup(data):
    #Create a new column of unique names so we can average them and find SE
    data['group']=data['id'].astype('str') + '-' + data['dpt'].astype('str')
    
    #Create new dataframe that summarizes entries in to means and std_error

    #Identify columns to aggregate
    cols=['anthos','spp','lpp','tannins','phenolics','non_tannin']

    #Create new dataframe, aggregate columns with summary statistics 'mean', 'stdev', and 'counts'
    means=pd.DataFrame()
    means=data.groupby('group')[cols].agg(['mean','std','count']).reset_index()

    #Get rid of the multi-level indexing
    means.columns=['_'.join(col).strip() for col in means.columns.values]

    #Add standard error columns
    for col in cols:
        means[f'{col}_SE']=means[f'{col}_std']/np.sqrt(means[f'{col}_count'])

    #Get rid of underscore in 'group' column
    means=means.rename(columns={'group_':'group'})

    #split the groups back up in to treatment, ph, and days post-treatment
    means[['trt','ph','dpt']]=means['group'].str.split('-', expand=True)

    #Make sure pH's are numerical values and not strings
    means['ph']=pd.to_numeric(means['ph'])

    return(means)

In [12]:
#Read CS data again, with unique name
cs_data=pd.read_csv("../01-input_data/tannin_data-cs.csv")

In [13]:
#Process using new function
cs_summ=pd.DataFrame()
cs_summ=cleanup(cs_data)

In [14]:
#Drop DPT = 0 datapoints
cs_summ=cs_summ[cs_summ['dpt'] != '0']

In [15]:
#Check to see if it came out the same
cs_summ.head(5)

,group,anthos_mean,anthos_std,anthos_count,spp_mean,spp_std,spp_count,lpp_mean,lpp_std,lpp_count,...,non_tannin_count,anthos_SE,spp_SE,lpp_SE,tannins_SE,phenolics_SE,non_tannin_SE,trt,ph,dpt
1,ACET-3.25-161,0.000000,0.000000,3,1.126667,0.092376,3,3.906667,0.040415,3,...,3,0.000000,0.053333,0.023333,9.504385,27.388156,18.520259,ACET,3.25,161
2,ACET-3.25-282,31.333333,5.131601,3,1.126667,0.156312,3,2.200000,0.202237,3,...,3,2.962731,0.090247,0.116762,10.535654,7.094599,8.647415,ACET,3.25,282
3,ACET-3.25-529,34.333333,6.658328,3,1.406667,0.106927,3,1.386667,0.204042,3,...,3,3.844188,0.061734,0.117804,3.844188,24.090109,25.141820,ACET,3.25,529
5,ACET-3.50-161,0.000000,0.000000,3,1.016667,0.155671,3,3.893333,0.381357,3,...,3,0.000000,0.089876,0.220177,13.715360,47.437678,37.687310,ACET,3.50,161
6,ACET-3.50-282,34.666667,11.846237,3,1.100000,0.310966,3,2.333333,0.094516,3,...,3,6.839428,0.179536,0.054569,9.237604,10.493384,12.142670,ACET,3.50,282


In [16]:
cs_summ==means

,group,anthos_mean,anthos_std,anthos_count,spp_mean,spp_std,spp_count,lpp_mean,lpp_std,lpp_count,...,non_tannin_count,anthos_SE,spp_SE,lpp_SE,tannins_SE,phenolics_SE,non_tannin_SE,trt,ph,dpt
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
10,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
11,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
13,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [17]:
#Looks like it worked

### Clean up CS and PT datasets, pickle data for later

In [20]:
cs_data=pd.read_csv("../01-input_data/tannin_data-cs.csv")
cs_summ=pd.DataFrame()
cs_summ=cleanup(cs_data)
cs_summ=cs_summ[cs_summ['dpt'] != '0']
cs_summ.to_pickle("../99-outputs/pickles/cs_data.pkl")

In [21]:
pv_data=pd.read_csv("../01-input_data/tannin_data-pv.csv")
pv_summ=pd.DataFrame()
pv_summ=cleanup(pv_data)
pv_summ=pv_summ[pv_summ['dpt'] != '0']
pv_summ.to_pickle("../99-outputs/pickles/pv_data.pkl")